Using R and TCGABiolinks we download the miRNA expressions for the TCGA-TGCT study.

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("TCGAbiolinks", ask = FALSE, verbose = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.0 (2025-04-11)

Installing package(s) 'BiocVersion', 'TCGAbiolinks'

system (cmd0): /usr/lib/R/bin/R CMD INSTALL

also installing the dependencies ‘plogr’, ‘png’, ‘Biostrings’, ‘RSQLite’, ‘KEGGREST’, ‘filelock’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘matrixStats’, ‘abind’, ‘SparseArray’, ‘AnnotationDbi’, ‘BiocFileCache’, ‘BiocGenerics’, ‘GenomeInfoDb’, ‘XVector’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘Biobase’, ‘S4Arrays’, ‘DelayedArray’, ‘downloader’, ‘biomaRt’, ‘GenomicRanges’, ‘XML’, ‘plyr’, ‘IRanges’, ‘S4Vectors’, ‘R.utils’, ‘SummarizedExperiment’, ‘TCGAbiolinksGUI.data’


foundpkgs: plogr, png, Biostrings, RSQLite, KEGGREST, filelock, UCSC.utils, GenomeInfo

In [26]:
library(SummarizedExperiment)

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins

In [1]:
library(TCGAbiolinks)
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [15]:
query_miRNA <- GDCquery(
  project = "TCGA-TGCT",
  data.category = "Transcriptome Profiling",
  data.type = "Isoform Expression Quantification",
)

--------------------------------------

o GDCquery: Searching in GDC database

--------------------------------------

Genome of reference: hg38

--------------------------------------------

oo Accessing GDC. This might take a while...

--------------------------------------------

ooo Project: TCGA-TGCT

--------------------

oo Filtering results

--------------------

ooo By data.type

----------------

oo Checking data

----------------

ooo Checking if there are duplicated cases

ooo Checking if there are results for the query

-------------------

o Preparing output

-------------------



In [16]:
GDCdownload(query_miRNA)


GDCdownload will download 156 files. A total of 86.396131 MB




Downloading: 9.6 MB     

In [17]:
data <- GDCprepare(query_miRNA)

  |======================================================================| 100%


In [28]:
head(data)

miRNA_ID,isoform_coords,read_count,reads_per_million_miRNA_mapped,cross-mapped,miRNA_region,barcode
<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>
hsa-let-7a-1,hg38:chr9:94175942-94175962:+,2,0.304530,N,precursor,TCGA-2G-AAF1-01A-11R-A439-13
hsa-let-7a-1,hg38:chr9:94175961-94175982:+,7,1.065854,N,"mature,MIMAT0000062",TCGA-2G-AAF1-01A-11R-A439-13
hsa-let-7a-1,hg38:chr9:94175961-94175983:+,4,0.609059,N,"mature,MIMAT0000062",TCGA-2G-AAF1-01A-11R-A439-13
hsa-let-7a-1,hg38:chr9:94175961-94175984:+,10,1.522648,N,"mature,MIMAT0000062",TCGA-2G-AAF1-01A-11R-A439-13
hsa-let-7a-1,hg38:chr9:94175962-94175981:+,413,62.885382,N,"mature,MIMAT0000062",TCGA-2G-AAF1-01A-11R-A439-13
hsa-let-7a-1,hg38:chr9:94175962-94175982:+,12346,1879.861817,N,"mature,MIMAT0000062",TCGA-2G-AAF1-01A-11R-A439-13


In [34]:
data_summarized <- data %>%
  mutate(miRNA_id_and_region = paste(miRNA_ID, miRNA_region, sep = ", "))

In [35]:
aggregated_data_rpm <- data_summarized %>%
  group_by(miRNA_id_and_region, barcode) %>%
  summarise(total_reads_per_million_miRNA_mapped = sum(reads_per_million_miRNA_mapped), .groups = "drop")

In [39]:
final_aggregated_data_rpm <- aggregated_data_rpm %>%
  pivot_wider(names_from = barcode, values_from = total_reads_per_million_miRNA_mapped)

In [40]:
head(final_aggregated_data_rpm)

miRNA_id_and_region,TCGA-2G-AAEW-01A-11R-A439-13,TCGA-2G-AAEX-01A-11R-A439-13,TCGA-2G-AAF1-01A-11R-A439-13,TCGA-2G-AAF4-01A-11R-A439-13,TCGA-2G-AAF6-01A-11R-A439-13,TCGA-2G-AAF8-01A-11R-A439-13,TCGA-2G-AAFE-01A-11R-A439-13,TCGA-2G-AAFG-01A-11R-A439-13,TCGA-2G-AAFG-05A-11R-A439-13,⋯,TCGA-YU-AA4L-01A-11R-A43A-13,TCGA-YU-AA61-01A-11R-A43A-13,TCGA-ZM-AA05-01A-12R-A43A-13,TCGA-ZM-AA06-01A-12R-A43A-13,TCGA-ZM-AA0B-01A-11R-A43A-13,TCGA-ZM-AA0D-01A-11R-A43A-13,TCGA-ZM-AA0E-01A-12R-A43A-13,TCGA-ZM-AA0F-01A-21R-A43A-13,TCGA-ZM-AA0H-01A-11R-A43A-13,TCGA-ZM-AA0N-01A-21R-A43A-13
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"hsa-let-7a-1, mature,MIMAT0000062",6619.132919,7412.830012,7711.910036,6090.723837,8102.343665,5459.291185,7449.903730,6551.645084,9446.725221,⋯,2832.443917,9104.099446,6668.826594,5451.417069,5148.370818,6938.135815,3701.072497,8413.282384,5926.730879,8547.070595
"hsa-let-7a-1, mature,MIMAT0004481",4.294653,3.647631,5.938329,6.816354,6.972757,7.579871,8.014657,4.291940,10.525468,⋯,3.661307,12.330930,8.466788,4.913399,7.076356,9.688443,9.122140,6.207074,11.366958,7.440915
"hsa-let-7a-1, precursor",0.671040,0.561174,0.304530,0.929503,0.410162,1.337624,0.745550,NA,0.467799,⋯,0.563278,0.237133,0.214349,NA,0.428870,0.605528,NA,0.269874,0.252600,0.544457
"hsa-let-7a-1, stemloop",NA,NA,NA,NA,NA,0.445875,0.186387,NA,0.233899,⋯,NA,0.316177,0.214350,NA,0.214435,0.151382,0.471835,0.089958,NA,NA
"hsa-let-7a-2, mature,MIMAT0000062",6651.745435,7454.356898,7686.786336,6134.410483,8174.122047,5449.927814,7403.493271,6495.849874,9354.334994,⋯,2854.975041,9138.167594,6618.240226,5416.102006,5147.727513,6837.012697,3620.860572,8343.205428,5898.060892,8565.037680
"hsa-let-7a-2, mature,MIMAT0010195",1.744703,2.525284,1.370384,3.408178,0.820324,1.783500,2.423036,2.861293,5.145785,⋯,NA,12.251886,4.072632,7.063012,1.929915,1.362437,1.572782,4.138050,3.788986,3.992685


In [41]:
write.csv2(final_aggregated_data_rpm, "TCGA_TGCT_mirna_isoform_data_rpm.csv", row.names = FALSE)

In [48]:
aggregated_data_raw_counts <- data_summarized %>%
  group_by(miRNA_id_and_region, barcode) %>%
  summarise(total_read_counts = sum(read_count), .groups = "drop")

In [50]:
final_aggregated_data_raw_counts <- aggregated_data_raw_counts %>%
  pivot_wider(names_from = barcode, values_from = total_read_counts)

In [52]:
head(final_aggregated_data_raw_counts)

miRNA_id_and_region,TCGA-2G-AAEW-01A-11R-A439-13,TCGA-2G-AAEX-01A-11R-A439-13,TCGA-2G-AAF1-01A-11R-A439-13,TCGA-2G-AAF4-01A-11R-A439-13,TCGA-2G-AAF6-01A-11R-A439-13,TCGA-2G-AAF8-01A-11R-A439-13,TCGA-2G-AAFE-01A-11R-A439-13,TCGA-2G-AAFG-01A-11R-A439-13,TCGA-2G-AAFG-05A-11R-A439-13,⋯,TCGA-YU-AA4L-01A-11R-A43A-13,TCGA-YU-AA61-01A-11R-A43A-13,TCGA-ZM-AA05-01A-12R-A43A-13,TCGA-ZM-AA06-01A-12R-A43A-13,TCGA-ZM-AA0B-01A-11R-A43A-13,TCGA-ZM-AA0D-01A-11R-A43A-13,TCGA-ZM-AA0E-01A-12R-A43A-13,TCGA-ZM-AA0F-01A-21R-A43A-13,TCGA-ZM-AA0H-01A-11R-A43A-13,TCGA-ZM-AA0N-01A-21R-A43A-13
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
"hsa-let-7a-1, mature,MIMAT0000062",49320,26419,50648,19658,19754,12244,39970,18318,40388,⋯,10057,115177,62224,17752,24009,45832,23532,93525,46926,47095
"hsa-let-7a-1, mature,MIMAT0004481",32,13,39,22,17,17,43,12,45,⋯,13,156,79,16,33,64,58,69,90,41
"hsa-let-7a-1, precursor",5,2,2,3,1,3,4,NA,2,⋯,2,3,2,NA,2,4,NA,3,2,3
"hsa-let-7a-1, stemloop",NA,NA,NA,NA,NA,1,1,NA,1,⋯,NA,4,2,NA,1,1,3,1,NA,NA
"hsa-let-7a-2, mature,MIMAT0000062",49563,26567,50483,19799,19929,12223,39721,18162,39993,⋯,10137,115608,61752,17637,24006,45164,23022,92746,46699,47194
"hsa-let-7a-2, mature,MIMAT0010195",13,9,9,11,2,4,13,8,22,⋯,NA,155,38,23,9,9,10,46,30,22


In [53]:
write.csv2(final_aggregated_data_raw_counts, "TCGA_TGCT_mirna_isoform_data_raw_counts.csv", row.names = FALSE)